In [134]:
import pandas as pd
import numpy as np
import unicodedata
import difflib
import unidecode

In [135]:
platforms = ["Guardian", "NME1", "Pitchfork", "Spectrum", "Uncut", "UTR", "Skinny"]
paths = [f"C:\\Users\\tommy\\OneDrive\\Third Year Project\\Platform Album Data\\{platform}.csv" for platform in platforms]

In [136]:
dfs = {platform: pd.read_csv(path, index_col=0) for platform, path in zip(platforms, paths)}

In [137]:
{platform: df.shape[0] for platform, df in dfs.items()}

{'Guardian': 183479,
 'NME1': 5052,
 'Pitchfork': 25396,
 'Spectrum': 7933,
 'Uncut': 10969,
 'UTR': 4200,
 'Skinny': 3734}

In [152]:
dfs = {platform: df.drop_duplicates() for platform, df in dfs.items()}

for platform, df in dfs.items():
    
    df.loc[:, "Album"] = df["Album"].apply(lambda x: unidecode.unidecode(unicodedata.normalize("NFKD", str(x))))
    df.loc[:, "Artist"] = df["Artist"].apply(lambda x: unidecode.unidecode(unicodedata.normalize("NFKD", str(x))))
    df.loc[:, "Album"] = df["Album"].apply(lambda x: x.strip("'`").replace(".", "").replace(",", "").replace("–", "").replace("-", "").replace(
        "&", "And").lower().strip())
    df.loc[:, "Artist"] = df["Artist"].apply(lambda x: x.strip("'`").replace(".", "").replace(",", "").replace("–", "").replace("-", "").replace(
        "&", "And").lower().strip())
    dfs[platform] = df
    
for platform, df in dfs.items():
    
    df.to_csv(f"C:\\Users\\tommy\\OneDrive\\Third Year Project\\Platform Album Data\\{platform}mod.csv")

In [153]:
df = dfs["Guardian"]

for key in list(dfs.keys())[1:4]:
    
    df = pd.merge(df, dfs[key], how="inner", on=["Album", "Artist"])
    print(df.shape[0])
    
df

1441
999
638


C:\Users\tommy\AppData\Local\Temp\ipykernel_17956\1547765782.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Platform_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, dfs[key], how="inner", on=["Album", "Artist"])


,Platform_x,Artist,Album,Platform_y,Platform_x,Platform_y
0,Guardian,bjork,fossora,NME,Pitchfork,Spectrum
1,Guardian,yeah yeah yeahs,cool it down,NME,Pitchfork,Spectrum
2,Guardian,alex g,god save the animals,NME,Pitchfork,Spectrum
3,Guardian,mura masa,demon time,NME,Pitchfork,Spectrum
4,Guardian,rina sawayama,hold the girl,NME,Pitchfork,Spectrum
...,...,...,...,...,...,...
633,Guardian,yuck,yuck,NME,Pitchfork,Spectrum
634,Guardian,pj harvey,let england shake,NME,Pitchfork,Spectrum
635,Guardian,kanye west,my beautiful dark twisted fantasy,NME,Pitchfork,Spectrum
636,Guardian,arctic monkeys,humbug,NME,Pitchfork,Spectrum


In [160]:
pd.merge(dfs["Guardian"], dfs["NME1"], how="inner", on=["Album", "Artist"])

,Platform_x,Artist,Album,Platform_y
0,Guardian,arctic monkeys,the car,NME
1,Guardian,brian eno,foreverandevernomore,NME
2,Guardian,the 1975,being funny in a foreign language,NME
3,Guardian,broken bells,into the blue,NME
4,Guardian,flohio,out of heart,NME
...,...,...,...,...
95,Guardian,mitski,laurel hell,NME
96,Guardian,cate le bon,pompeii,NME
97,Guardian,alice glass,prey//iv,NME
98,Guardian,mo,motordrome,NME


In [157]:
nme = dfs["NME1"].copy()
g = dfs["Guardian"].copy()

nme["Full"] = nme["Album"] + " " + nme["Artist"]
g["Full"] = g["Album"] + " " + g["Artist"]

nme["Matches"] = nme["Full"].apply(lambda x: difflib.get_close_matches(x, g["Full"][:1000], cutoff=0.9))

nme.loc[nme["Matches"].apply(lambda x: len(x)) != 0]

,Platform,Artist,Album,Full,Matches
0,NME,arctic monkeys,the car,the car arctic monkeys,[the car arctic monkeys]
10,NME,brian eno,foreverandevernomore,foreverandevernomore brian eno,[foreverandevernomore brian eno]
12,NME,the 1975,being funny in a foreign language,being funny in a foreign language the 1975,[being funny in a foreign language the 1975]
14,NME,willow,copingmechanism,copingmechanism willow,[coping mechanism willow]
15,NME,flohio,out of heart,out of heart flohio,[out of heart flohio]
...,...,...,...,...,...
1096,NME,stephen malkmus,traditional techniques,traditional techniques stephen malkmus,[traditional techniques stephen malkmus]
1097,NME,us girls,heavy light,heavy light us girls,[heavy light us girls]
1099,NME,cornershop,england is a garden,england is a garden cornershop,[england is a garden cornershop]
2393,NME,run the jewels,run the jewels 2,run the jewels 2 run the jewels,[run the jewels 4 run the jewels]


In [165]:
difflib.get_close_matches("demon time", dfs["Guardian"].loc[:, "Album"])

['demon time', 'down time', 'demoni']